In [38]:
from math import *
def daylight(lat,J):
    
    """lat  latitude [degree]
        J  Julian day
      return L (daylight hours in units of 12 hours), w ( sunset hour angle [radians], d (declination [radians] )"""
    dec=0.4093*sin(2*pi/365*J-1.39) #d declination [radians]
    t=tan(dec)*tan(radians(float(lat)))
    w=acos(-t)#w   the sunset hour angle [radians]
    L=24/pi*w #L daylight hours in units of 12 hours (Average day length of the month )
    return L,w,dec

In [39]:
daylight(34,123)

(13.446604497114622, 1.7601564126609488, 0.2721397327082406)

In [42]:
def etp(T,lat):
    julian=[15,46,74,105,135,166,196,227,258,288,319,349] # mean average Julian day for each month
    days=[31,28,31,30,31,30,31,31,30,31,30,31]# days in months
    I= sum([(t/5)**1.514  if t>0 else 0 for t in T])#  S(Tm/5)**1.5;for month m:Thornthwaite's temperature efficiency index,heat index  for the year: I index for each month when temperature more than zero.
    a=(6.75e-07)*I**3 -(7.71e-05)*I**2 + 0.01792*I + 0.49239 # Expentail factor
    pet=[]
    for n,  t in  zip(days,T):
        if t<=0:
            pet.append(0)
        elif 0<t<=26.5:
            et=16 *(10*t/I)**a
            pet.append(et)
##            print(et,t)
        elif 26.5<t:
            et=-415.85+32.24*t-0.43*t**2
            pet.append(et)
        else:
            raise
    correction=[n/30*(daylight(lat,j)[0])/12. for n, j in zip(days,julian)]
    pet=[c*et for c,et in zip(correction ,pet)]
    return pet

In [24]:
def pascal():
    row=[1]
    while True:
        yield row
        row= [i+j for  i,j  in zip(row+[0],[0]+row)]
def binom(N,s,exact=1):
    if exact:
        if (s > N) or (N < 0) or (s < 0):
            return 0
    can=pascal()
    k=[next(can) for i in range(N+1)]
    return k[N][s]

In [25]:
binom(5,2)


10

In [26]:
def w(x,s):
    """X:data series
         s: order, integer value {0, 1, .....}
     return The unbiased PWMs (probability weighted moments)"""
    x=list(x)
    x.sort()
    n=len(x)
    s=int(s)
    ws=sum([1/n* x[i-1]*binom(n-i,s)/binom(n-1,s) for i in range(1, n+1)])
    return ws

In [27]:
def LLfit(x):
    """x:data series
        return: scale,shape , location 
        the parameters of the Log-logistic distribution after Singh et al. (1993) """
    w0,w1,w2=w(x,0),w(x,1),w(x,2)
    beta=(2*w1-w0)/(6*w1-w0-6*w2)
    gb=gamma(1+1/beta)*gamma(1-1/beta)
    alpha=(w0-2*w1)*beta/gb
    omga=w0-alpha*gb
    return  alpha,beta,omga #

In [28]:
def logLogisticCDF(x,alpha,beta,omga):
    """ x : non-negative  data series
      Cumulative distribution function of a three parameter Log-logistic distributin  (Fisk distribution)
        scale=.1
        shape=3
        loc=5
        x=7
        logLogisticCDF(x,scale, shape,loc)
        0.9998750156230471"""
    return 1 / (1+pow(alpha/(x-omga),beta));

In [29]:
scale=.1
shape=3
loc=5
x=7
logLogisticCDF(x,scale, shape,loc)

0.9998750156230471

In [58]:
from collections import Counter
def SPEI_function(x,fit=None):
    if fit==LLfit:
        cdf=logLogisticCDF  
    zeros=Counter(x)[0]
    pzero=zeros/len(x)
    x2=x[:]
    while 0 in x2 :
        x2.remove(0)
    alpha,beta,omga=fit(x2)
    print(str(fit.__name__)+" Fit: ",alpha,beta,omga)
    spei=[cdf2Z(pzero + (1.-pzero) *cdf(i,alpha,beta,omga)) for i in x]
    return spei

In [59]:
########################################################
####################################################
from collections import defaultdict
from itertools import groupby
def group(indata,indate,func=None, fit=None):
    zdata=list(zip(indata,indate))
    grp=groupby(zdata, lambda x: x[1].month)
    P=defaultdict(list)
    gdates= defaultdict(list)
    for i, g in grp:
            for v , d in g:
                    gdates[i].append(d)
                    P[i].append(v)
    res=defaultdict(list)
    for key  in P:
            spiout=func(P[key],fit=fit)
            kdates=gdates[key]
            for d in kdates:
                    res["date"].append(d)
            for v in spiout:
                res["spi"].append(v)
    zdata=list(zip(res["spi"],res["date"]))
    zdata.sort(key=lambda x :x[1])
    res.clear()
    for  k in zdata:
            res["spi"].append(k[0])
            res["date"].append(k[1])
    sp,dat=res["spi"],res["date"]
    return sp,dat

In [64]:
class SPEI (object):
    def __init__(self, date,rain,Tmean,latDeg=None, scales=None):
        self.rain, self.Tmean, self.date=rain,Tmean,date
        self.scales=scales
        self.Ds,self.dates,self.ETP=self.initial(latDeg)
    def initial(self,latDeg):
        values=zip(self.Tmean, self.date,self.rain)
        grp=groupby(values, key=lambda x : x[1].year)
        data=defaultdict(list)
        for k, g in  grp:
            g=list(g)
            t=[i[0] for i in g]
            date=[i[1] for i in g]
            etp1=etp(t, latDeg)
            #print(etp1)
            D=[g[i][2]-etp1[i] for i in range(len(g))]
            for i in range(len(etp1)):
                data["etp"].append(etp1[i])
                data["D"].append(D[i])
                data["date"].append(g[i][1])
        return  data["D"],data["date"],data["etp"]
    def calculate(self, fit=None):
        speiData=defaultdict(list)
        speiData["date"]=self.dates
        for scale in self.scales:
            print("scale\t",scale)
            data=summov(self.Ds,scale)
            #print(data)
            dates2=self.dates[scale-1:]
            spei,groupdate=group(data,dates2,func=SPEI_function, fit=fit)
            lags= [float("nan") for i in range(scale-1)]
            for i in lags+spei:
                speiData[scale].append(i)
        return speiData

In [48]:
def main():
    import sys
    if  len(sys.argv) >=7:
        script = sys.argv[0]
        infile = sys.argv[1]
        rainCol ,Tcol,dateCol= sys.argv[2], sys.argv[3],sys.argv[4]
        lat, outfile ,scales= sys.argv[5], sys.argv[6],sys.argv[7:]
        scales=map(int,scales)
        data=readcsv(infile)
        rain, tmean, date=data[rainCol],data[Tcol],data[dateCol]
        myspei=SPEI(date,rain, tmean, latDeg=lat ,scales=scales)
        myspeic=myspei.calculate(fit=util.LLfit)
        print(myspeic.keys())
        util.write(myspeic,outfile)
        print ('spei is calcualted and saved in {}'.format(outfile))
    else:
        print ("\nUsage: spei  <infile> <RAIN col> <Tmean col> <date col>\
<latitude> <outfile> <scales>\n")
        print ("Example: spei.py  data/wichita.csv  PRCP TMED date 37.6475 data/outspei.csv 1 3 6 9 12 24")
        #sys.exit(2)
if __name__ == '__main__':
    main()


Usage: spei  <infile> <RAIN col> <Tmean col> <date col><latitude> <outfile> <scales>

Example: spei.py  data/wichita.csv  PRCP TMED date 37.6475 data/outspei.csv 1 3 6 9 12 24


In [54]:
##################################فصل 15 شکل  #################################
from collections import defaultdict
import io,csv
def readcsv(infile):
    """input file name, csv format"""
    with io.open(infile, newline='') as csvfile:
            data = defaultdict(list)
            reader = csv.DictReader(csvfile)
            for row in reader:
                for key in  row:
                    try:
                        data[key].append(dateparser(row[key]))
                    except:
                        if row[key]=="NA":
                            row[key]=float("nan")
                        data[key].append(float(row[key]))  
    return data
##################################فصل 15 شکل  ##############################
from datetime import datetime
dateparser=lambda x:datetime.strptime(x,
                                      "%Y-%m-%d") if "-" in x else datetime.strptime(x,
                                                                                     "%m/%d/%Y")
#############################################

def cdf2Z(p):
    c=p
    """ P probably betwen 0 and 1 """
    if p<=0 or p>1 :
        print("p={} is out of range".format(c))
        return float("inf")
    if (c-0.5)>0:
        c=1. - c
    if (c-0.5)<0:
        #print(c)
        dt2= -2*log(c)
        t=sqrt(dt2)
        ppf= t- ((.010328*t + .802853)*t+ 2.515517) /\
             (((.001308*t + .189269)*t + 1.432788)*t + 1.)
        if p<0.5:
            ppf=-ppf
        return ppf
    if c-0.5==0:
        return 0
###########################################
summov=lambda x,win:[sum(x[i:win+i])for i in range(len(x[win-1:]))]
###################################################
def write (data, outfile, fields=None):
    with io.open(outfile, 'w', newline='') as csvfile:
        if fields==None:
            fields=list(data.keys())
            print(fields)
        csvWriter = csv.DictWriter(csvfile, fieldnames=fields)
        csvWriter.writeheader()
    with io.open(outfile, 'a', newline='') as csvfile:
        writer = csv.writer(csvfile, delimiter=',', quotechar='"',
                            quoting=csv.QUOTE_MINIMAL)
        for  row in list(zip(*data.values())):
            row=list(row)
            row[0]=row[0].strftime("%Y-%m-%d")
            #print(row)
            writer.writerow(row)
##            if not row:
##                print("empty")
    print("The result is saved as {}".format(outfile))

In [66]:
rainCol ,Tcol,dateCol= "PRCP", "TMED","date"

lat, outfile ,scales= 37, "D:/test_spei.csv",[1,3,6,9,12]
scales=map(int,scales)
infile="D:/spi/pydrought/data/wichita.csv"
data=readcsv(infile)
rain, tmean, date=data[rainCol],data[Tcol],data[dateCol]
myspei=SPEI(date,rain, tmean, latDeg=lat ,scales=scales)
myspeic=myspei.calculate(fit=LLfit)
print(myspeic.keys())
write(myspeic,outfile)

scale	 1
LLfit Fit:  50.212080853093326 5.076236857376864 -33.13226354887902
LLfit Fit:  81.76115906069602 5.786497883994734 -60.544976876441964
LLfit Fit:  139.0876380844144 6.059680220484953 -98.71223667631159
LLfit Fit:  110.87546162220731 4.302732043104933 -107.90538676817411
LLfit Fit:  155.50493774406414 4.396371756029603 -152.08315860646871
LLfit Fit:  -1555.4270964816849 -36.39931962722487 1540.7852164299945
LLfit Fit:  -3236.294173469918 -96.67229513797973 3138.8929939031514
LLfit Fit:  271.96163082882435 7.0603797017956085 -347.9790983336559
LLfit Fit:  80.66218774875256 2.646034250664565 -127.96216240117508
LLfit Fit:  119.33430394318745 4.170630287421652 -114.16352901023629
LLfit Fit:  67.53317457838122 4.179782717737889 -54.61636431460734
LLfit Fit:  40.513951714675265 3.2304967491940793 -18.27988294609041
scale	 3
LLfit Fit:  381.2356392325267 10.685834755374279 -294.1664857337744
LLfit Fit:  398.44725153354136 9.64875244566981 -319.9222519942308
LLfit Fit:  574.081064344